# GPTChallenge: diagnóstico a partir de HCE

Vamos a trabajar con el corpus CodEsp (textos de historial clínico etiquetados con sus códigos CIE-10 Diagnóstico)

In [1]:
import pandas as pd
import os, re, spacy
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier, SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, make_scorer, f1_score
from sklearn.metrics.pairwise import cosine_similarity
import lightgbm as lgb
from scipy.sparse import vstack

pd.options.display.max_colwidth = None

In [2]:
def normalizeDoc(nlp, doc, nMinCharacters = 0):
    """
    Normaliza un texto eliminando palabras por debajo del mínimo de caracteres, stop words y números.
    Para ello, tokeniza empleando un modelo de Spacy.
    """
    # Separar en tokens
    tokens = nlp(doc)
    # Filtrar tokens
    filtered_tokens = [t.lower_ for t in tokens if (len(t.text) >= nMinCharacters) and not t.is_punct and not re.match('[0-9]+', t.text)]
    # Recombinamos los tokens
    doc = ' '.join(filtered_tokens)
    
    return doc

def findMostSimilar(similarityDf: pd.DataFrame, data: pd.DataFrame, nMostSimilars: int = 1):
    """
    Encuentra la etiqueta de los documentos más similares.
    """
    # Crear df de resultados
    results = pd.DataFrame(index = similarityDf.index, columns = ['archivoMostSimilar', 'similarity', 'codigosPred'])

    for index, row in similarityDf.iterrows():
        
        # Buscar la máxima similitud
        mostSimilar = row.nlargest(nMostSimilars)
        
        results.loc[index, 'archivoMostSimilar'] = mostSimilar.index.values
        if nMostSimilars == 1:
            results.loc[index, 'similarity'] = mostSimilar.values[0]
        else:
            results.loc[index, 'similarity'] = row[mostSimilar.index]

    # Coger las etiquetas
    if nMostSimilars == 1:
        results['codigosPred'] = data.loc[np.squeeze(np.vstack(results['archivoMostSimilar'].values)), 'codigos'].values
    else:
        pass
    return results

def checkAccuracy(pred: pd.DataFrame, data: pd.DataFrame):
    pred['codigos'] = data.loc[pred.index, 'codigos']

    pred['guess'] = pred.apply(
        lambda row: len(set(row['codigosPred']).intersection(row['codigos'])) / len(row['codigosPred']),
        axis = 1)
    return pred

## Lectura y preprocesamiento de los datos

### Conjunto de train

In [3]:
#los códigos están en un TSV con un código por línea
train_diag = pd.read_csv("data/train/train.tsv", sep="\t", header=None, names=["archivo", "codigo"])
train_diag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8316 entries, 0 to 8315
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   archivo  8316 non-null   object
 1   codigo   8316 non-null   object
dtypes: object(2)
memory usage: 130.1+ KB


In [4]:
#cogemos la categoría superior de cada código y las agrupamos
train_diag['cat'] = train_diag['codigo'].str.extract(r'(\w\d\d)')
print(train_diag['cat'].value_counts())
train_diag['cat'].nunique()

cat
r52    163
r10    163
r59    160
r69    150
r50    144
      ... 
c31      1
d62      1
s53      1
s34      1
n81      1
Name: count, Length: 918, dtype: int64


918

In [5]:
categories=train_diag['cat'].value_counts()[:10]
top_categorias = categories.index.to_list()
print(top_categorias)

['r52', 'r10', 'r59', 'r69', 'r50', 'r60', 'i10', 'r11', 'n28', 'd49']


In [6]:
#seleccionamos sólo las etiquetas de este subconjunto
train_diag = train_diag[np.isin(train_diag['cat'], top_categorias)]

In [7]:
#cargamos los dos conjuntos de train
path = 'data/train/text_files/'

corpus = []
for f in [f for f in os.listdir(path) if f.endswith('.txt')]:
    with open(os.path.join(path, f), encoding="utf8") as text:
        texto = text.read()
    #buscamos códigos
    file = f[:-4]
    codigos = train_diag.query('archivo==@file')['cat'].to_list()
    codigos = list(set(codigos))
    if codigos:
        corpus.append({
            'archivo': file,
            'texto': texto,
            'codigos': codigos
        })
    
df_train = pd.DataFrame(corpus).set_index('archivo')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 562 entries, S0004-06142005000700014-1 to S2340-98942015000100005-1
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   texto    562 non-null    object
 1   codigos  562 non-null    object
dtypes: object(2)
memory usage: 13.2+ KB


In [8]:
df_train.sample(1)

,texto,codigos
archivo,,
S0365-66912011000600005-2,"Varón de 37 años ex-adicto a drogas por vía parenteral y diagnosticado de hepatitis crónica por virus C genotipo 4 con una baja carga viral (75.754 UI/ml). Se instauró tratamiento con IFN pegilado alpha-2b (100µg/semana) y ribavirina (1g/día).\nA los 2 meses del inicio del tratamiento acudió a la consulta y refirió visión borrosa y «manchas» en el campo visual desde aproximadamente 10 días antes. En la exploración oftalmológica presentó una AV de 0,8 con corrección en ambos ojos y múltiples exudados algodonosos en polo posterior. Se diagnosticó de retinopatía por IFN y se suspendió el tratamiento. En controles sucesivos el cuadro fue mejorando y a los 3 meses presentó una AV de 10/10 en ambos ojos y en el FO únicamente se apreció un exudado algodonoso en reabsorción inferior a fóvea izquierda.\n\n",[r60]


### Conjunto de test

In [9]:
#los códigos están en un TSV con un código por línea
test_diag = pd.read_csv("data/test/test.tsv", sep = "\t", header = None, names = ["archivo"])
test_diag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   archivo  192 non-null    object
dtypes: object(1)
memory usage: 1.6+ KB


In [10]:
path = 'data/test/text_files/'

corpus = []
for f in [f for f in os.listdir(path) if f.endswith('.txt')]:
    file = os.path.splitext(f)[0]
    if file in test_diag['archivo'].values:
        with open(os.path.join(path, f), encoding="utf8") as text:
            texto = text.read()
            corpus.append({
                'archivo': file,
                'texto': texto
            })
    
df_test = pd.DataFrame(corpus).set_index('archivo')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 192 entries, S0004-06142005000500011-1 to S2254-28842014000300010-1
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   texto   192 non-null    object
dtypes: object(1)
memory usage: 3.0+ KB


In [11]:
df_test.sample(1)

,texto
archivo,
S1887-85712012000400006-1,"Mujer de 47 años que consulta en urgencias por un cuadro de 8 horas de evolución con dolor abdominal continuo y difuso, junto con vómitos, primero alimentarios y posteriormente biliosos. La paciente tiene únicamente como antecedentes personales, la realización de dos cesáreas. En urgencias presenta una tensión arterial de 122/85 mmHg, una frecuencia cardíaca de 91 latidos por minuto y una temperatura axilar de 35.9 oC. La exploración abdominal revela una cicatriz de laparotomía media infraumbilical, la presencia de ruidos disminuidos, y dolor a la palpación de manera difusa sin claros signos de irritación peritoneal. No existen hernias inguinales o crurales.\nEn el análisis de sangre destacan tan solo una glucemia de 122 mg/dL, una leucocitosis de 15.240 uL, junto con neutrofília del 93%, siendo el resto de parámetros normales. La radiografía simple de abdomen es compatible con suboclusión de intestino delgado, y colelitiasis. La TAC de urgencias revela la presencia de asas de intestino delgado dilatadas y sugiere la posibilidad de una torsión intestinal.\n\nLos hallazgos en las exploraciones complementarias, junto con el empeoramiento clínico de la paciente durante el periodo de observación y estudio, aumentando los vómitos y el dolor abdominal, así como apareciendo signos de irritación peritoneal en hemiabdomen inferior, hacen que se le proponga una laparoscópia exploradora. La sospecha diagnóstica en ese momento es de oclusión intestinal por bridas, y el tiempo trancurrido desde el ingreso en urgencias, de 16 h.\nBajo anestesia general se realiza laparoscópia exploradora, identificando en la pelvis de la paciente un asa de intestino delgado estrangulada y necrosada, este asa ocupa la derecha de la figura 3 (color morado/negro), la cual es una fotografía intraoperatoria directa del monitor de la cámara laparoscópica. La parte izquierda de esa imagen se corresponde con el ileon pre-herniario (de color rosa normal), entre ambas zonas de ileon está la trompa de Falopio derecha la cual está adherida a la pared abdominal anterior y bajo ella se extiende el ligamento ancho (no visible en la imagen) y que tendría el orificio a través del cual se produce la herniación, en la figura 4 el autor representa un dibujo esquemático para comprender mejor la anatomía de los hallazgos intraoperatorios. La gran dilatación de asas de intestino delgado, disminuye mucho el espacio de trabajo necesario para manipular el instrumental quirúrgico por laparoscópia. Ese hecho, junto con el riesgo de perforación al manipular el asa necrosada, así como la imposibilidad de identificar con seguridad las estructuras anatómicas, obligan a una reconversión a cirugía abierta. La reconversión se realiza mediante una laparotomía infraumbilical iterativa y se aprecia que la causa de la obstrucción y estrangulación del ileon, es una hernia interna a través de la hoja derecha del ligamento ancho del útero. Se procede a la reducción de la hernia, resección de 25 cm de ileon necrosado y reconstrucción con anastomosis termino-terminal manual. Sobre el orificio herniario se realiza el cierre con sutura continua de seda 0/0. El postoperatorio transcurre con normalidad y la paciente es dada de alta con buen estado clínico a los 11 días del ingreso. El control postoperatorio al mes es correcto.\n\n"


### Binarizar las etiquetas

In [12]:
# para entrenar un clasificador multi-etiqueta generamos una matriz binaria de las etiquetas
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(df_train['codigos'])

#Guardamos las clases utilizadas en el conjunto de train
clases = mlb.classes_
num_classes = clases.shape
print(num_classes[0])

10


## Procesamiento del lenguaje natural

### BoW

In [13]:
# python -m spacy download es_core_news_lg
nlp = spacy.load("es_core_news_lg")

Obtenemos resultados ligeramente mejores sin realizar un preprocesamiento con la función `normalizeDoc()` previamente a la aplicación del *Bag of Words*. La adición de *n-grams* empeora los resultados de los modelos.

In [14]:
# x_trainText = [normalizeDoc(nlp, doc) for doc in df_train['texto'].values]
x_trainText = df_train['texto'].values
x_testText = df_test['texto'].values

vectorizer = CountVectorizer()
x_trainArray = vectorizer.fit_transform(x_trainText)
x_testArray = vectorizer.transform(x_testText)

## Modelos

In [15]:
x_train, x_test, y_train_train, y_train_test = train_test_split(x_trainArray, y_train, test_size = 0.1, random_state = 3)

### Nuestro mejor modelo: MultiOutput con GradientBoosting

In [16]:
base_classifier = GradientBoostingClassifier(loss = 'exponential', n_estimators = 50, max_depth = 5, min_samples_split = 5, random_state = 3)
model = MultiOutputClassifier(base_classifier, n_jobs = 4)
model.fit(x_train, y_train_train)
y_pred = model.predict(x_test)
report = classification_report(y_train_test, y_pred, zero_division = 0)
print(report)

              precision    recall  f1-score   support

           0       0.89      0.62      0.73        13
           1       1.00      1.00      1.00        16
           2       1.00      0.75      0.86         8
           3       0.80      0.57      0.67        14
           4       1.00      1.00      1.00         8
           5       1.00      0.93      0.97        15
           6       1.00      0.31      0.47        13
           7       1.00      0.92      0.96        12
           8       0.88      0.88      0.88         8
           9       1.00      1.00      1.00        17

   micro avg       0.96      0.80      0.87       124
   macro avg       0.96      0.80      0.85       124
weighted avg       0.96      0.80      0.85       124
 samples avg       0.92      0.82      0.85       124



### Coseno similitud (*manual*)

In [28]:
xTag = df_train.index.values

x_trainT, x_testT, xTag_train, xTag_test = train_test_split(x_train, xTag, random_state = 3)

In [29]:
vectorizer = CountVectorizer()
x_trainVec = vectorizer.fit_transform(x_trainT)
x_trainVecDf = pd.DataFrame(x_trainVec.toarray(), columns = vectorizer.get_feature_names_out(), index = xTag_train)

x_testVec = vectorizer.transform(x_testT)
x_testVecDf = pd.DataFrame(x_testVec.toarray(), columns = vectorizer.get_feature_names_out(), index = xTag_test)

similarity = cosine_similarity(x_testVecDf, x_trainVecDf)
similarityDf = pd.DataFrame(similarity, index = x_testVecDf.index, columns = x_trainVecDf.index)

In [30]:
mostSimilarDf = findMostSimilar(similarityDf, df_train)

mostSimilarDf.head()

,archivoMostSimilar,similarity,codigosPred
S0210-48062010000100019-4,S0210-48062005000600013-1,0.339525,[n28]
S1134-80462015000200005-1,S0212-16112010000600024-1,0.269564,"[r10, r69, r11, r60]"
S1139-76322009000400007-1,S1137-66272013000300020-1,0.352673,[r10]
S0376-78922014000100013-1,S0376-78922016000200012-1,0.40826,"[r69, d49, i10]"
S1130-01082007001100012-1,S0212-16112010000100017-1,0.221398,[r60]


In [31]:
mostSimilarDf = checkAccuracy(mostSimilarDf, df_train)
print(f"Al menos 1 coincidencia: {(mostSimilarDf['guess'] != 0).sum() / mostSimilarDf.shape[0]}")
print(f"Accuracy media: {mostSimilarDf['guess'].mean()}")

Al menos 1 coincidencia: 0.624113475177305
Accuracy media: 0.4052178318135765


### Otros modelos de clasificación multietiqueta

#### Random Forest

In [66]:
multi_target_classifier = RandomForestClassifier(n_estimators = 10, criterion = 'gini', n_jobs = 4, random_state = 3)
multi_target_classifier.fit(x_train, y_train_train)
y_pred = multi_target_classifier.predict(x_test)
report = classification_report(y_train_test, y_pred, zero_division = 0)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       1.00      0.25      0.40        16
           2       0.00      0.00      0.00         8
           3       1.00      0.07      0.13        14
           4       1.00      0.25      0.40         8
           5       0.75      0.20      0.32        15
           6       0.33      0.08      0.12        13
           7       0.00      0.00      0.00        12
           8       0.00      0.00      0.00         8
           9       0.00      0.00      0.00        17

   micro avg       0.79      0.09      0.16       124
   macro avg       0.41      0.08      0.14       124
weighted avg       0.43      0.09      0.14       124
 samples avg       0.17      0.08      0.11       124



#### Multi Layer Perceptron

In [68]:
multi_target_classifier = MLPClassifier(activation = 'logistic', solver = 'adam', max_iter = 1000, random_state = 3)
multi_target_classifier.fit(x_train, y_train_train)
y_pred = multi_target_classifier.predict(x_test)
report = classification_report(y_train_test, y_pred, zero_division = 0)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.08      0.14        13
           1       0.80      0.25      0.38        16
           2       0.83      0.62      0.71         8
           3       1.00      0.29      0.44        14
           4       0.20      0.12      0.15         8
           5       0.75      0.40      0.52        15
           6       0.50      0.23      0.32        13
           7       0.83      0.42      0.56        12
           8       0.00      0.00      0.00         8
           9       0.80      0.24      0.36        17

   micro avg       0.70      0.27      0.39       124
   macro avg       0.67      0.26      0.36       124
weighted avg       0.72      0.27      0.37       124
 samples avg       0.38      0.25      0.29       124



#### K Neighbors

In [23]:
multi_target_classifier = KNeighborsClassifier(n_neighbors = 5)
multi_target_classifier.fit(x_train, y_train_train)
y_pred = multi_target_classifier.predict(x_test)
report = classification_report(y_train_test, y_pred, zero_division = 0)
print(report)

              precision    recall  f1-score   support

           0       0.50      0.08      0.13        26
           1       0.25      0.06      0.09        35
           2       0.33      0.23      0.27        13
           3       0.43      0.10      0.17        29
           4       0.50      0.04      0.08        23
           5       0.40      0.06      0.10        36
           6       0.39      0.23      0.29        31
           7       0.24      0.20      0.22        25
           8       0.67      0.08      0.14        25
           9       0.31      0.23      0.26        35

   micro avg       0.34      0.13      0.18       278
   macro avg       0.40      0.13      0.18       278
weighted avg       0.40      0.13      0.17       278
 samples avg       0.19      0.11      0.13       278



#### MultiOutput

##### Regression Classifiers

In [29]:
base_classifier = LogisticRegression(penalty = 'l2', solver = 'newton-cg', max_iter = 1000, multi_class = 'multinomial')
multi_target_classifier = MultiOutputClassifier(base_classifier, n_jobs = 4)
multi_target_classifier.fit(x_train, y_train_train)
y_pred = multi_target_classifier.predict(x_test)
report = classification_report(y_train_test, y_pred, zero_division = 0)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.27      0.42        26
           1       0.88      0.43      0.58        35
           2       0.56      0.69      0.62        13
           3       0.87      0.45      0.59        29
           4       0.78      0.30      0.44        23
           5       0.83      0.42      0.56        36
           6       0.53      0.26      0.35        31
           7       0.77      0.40      0.53        25
           8       0.86      0.24      0.38        25
           9       0.88      0.40      0.55        35

   micro avg       0.78      0.37      0.51       278
   macro avg       0.80      0.39      0.50       278
weighted avg       0.81      0.37      0.50       278
 samples avg       0.50      0.35      0.40       278



In [56]:
base_classifier = PassiveAggressiveClassifier(random_state = 3)
multi_target_classifier = MultiOutputClassifier(base_classifier, n_jobs = 4)
multi_target_classifier.fit(x_train, y_train_train)
y_pred = multi_target_classifier.predict(x_test)
report = classification_report(y_train_test, y_pred, zero_division = 0)
print(report)

              precision    recall  f1-score   support

           0       0.70      0.27      0.39        26
           1       0.79      0.54      0.64        35
           2       0.56      0.69      0.62        13
           3       0.78      0.48      0.60        29
           4       0.64      0.39      0.49        23
           5       0.83      0.53      0.64        36
           6       0.53      0.26      0.35        31
           7       0.79      0.44      0.56        25
           8       0.88      0.28      0.42        25
           9       0.89      0.49      0.63        35

   micro avg       0.75      0.43      0.55       278
   macro avg       0.74      0.44      0.53       278
weighted avg       0.75      0.43      0.54       278
 samples avg       0.55      0.44      0.46       278



In [72]:
base_classifier = SGDClassifier(loss = 'squared_hinge', penalty = 'elasticnet', random_state = 3)
multi_target_classifier = MultiOutputClassifier(base_classifier, n_jobs = 4)
multi_target_classifier.fit(x_train, y_train_train)
y_pred = multi_target_classifier.predict(x_test)
report = classification_report(y_train_test, y_pred, zero_division = 0)
print(report)

              precision    recall  f1-score   support

           0       0.50      0.27      0.35        26
           1       0.78      0.51      0.62        35
           2       0.69      0.69      0.69        13
           3       0.88      0.48      0.62        29
           4       0.58      0.48      0.52        23
           5       0.86      0.50      0.63        36
           6       0.41      0.29      0.34        31
           7       0.71      0.68      0.69        25
           8       0.58      0.28      0.38        25
           9       0.76      0.37      0.50        35

   micro avg       0.68      0.44      0.54       278
   macro avg       0.68      0.46      0.54       278
weighted avg       0.69      0.44      0.53       278
 samples avg       0.53      0.45      0.46       278



##### Random Forest

In [29]:
base_classifier = RandomForestClassifier(n_estimators = 10)
multi_target_classifier = MultiOutputClassifier(base_classifier, n_jobs = 4)
multi_target_classifier.fit(x_train, y_train_train)
y_pred = multi_target_classifier.predict(x_test)
report = classification_report(y_train_test, y_pred, zero_division = 0)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       1.00      0.31      0.48        16
           2       1.00      0.25      0.40         8
           3       1.00      0.14      0.25        14
           4       0.50      0.12      0.20         8
           5       1.00      0.33      0.50        15
           6       0.00      0.00      0.00        13
           7       0.67      0.17      0.27        12
           8       0.00      0.00      0.00         8
           9       0.80      0.24      0.36        17

   micro avg       0.88      0.17      0.28       124
   macro avg       0.60      0.16      0.25       124
weighted avg       0.63      0.17      0.26       124
 samples avg       0.27      0.15      0.18       124



##### Gradient Boosting: Grid Search (mejor modelo)

In [58]:
base_classifier = GradientBoostingClassifier(loss = 'exponential', random_state = 3)
multi_target_classifier = MultiOutputClassifier(base_classifier, n_jobs = 4)

param_grid = {
    'estimator__n_estimators': [40, 50, 60],
    'estimator__max_depth': [3, 5, 7],
    'estimator__min_samples_split': [2, 5, 10]
}

f1_scorer = make_scorer(f1_score, average = 'weighted')

grid_search = GridSearchCV(multi_target_classifier, param_grid, cv = 5, scoring = f1_scorer, n_jobs = 4, verbose = 1)
grid_search.fit(x_train, y_train_train)

best_params = grid_search.best_params_
print("Best parameters:", best_params)

best_estimator = grid_search.best_estimator_
best_estimator.fit(x_train, y_train_train)

y_pred = best_estimator.predict(x_test)

report = classification_report(y_train_test, y_pred, zero_division = 0)
print(report)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best parameters: {'estimator__max_depth': 5, 'estimator__min_samples_split': 5, 'estimator__n_estimators': 50}
              precision    recall  f1-score   support

           0       0.67      0.38      0.49        26
           1       1.00      1.00      1.00        35
           2       0.69      0.69      0.69        13
           3       0.87      0.69      0.77        29
           4       1.00      0.87      0.93        23
           5       1.00      0.94      0.97        36
           6       0.82      0.45      0.58        31
           7       1.00      0.92      0.96        25
           8       0.96      0.92      0.94        25
           9       1.00      1.00      1.00        35

   micro avg       0.93      0.80      0.86       278
   macro avg       0.90      0.79      0.83       278
weighted avg       0.92      0.80      0.85       278
 samples avg       0.88      0.80      0.82       278



Intentamos mejorar el modelo añadiendo un `AdaBoostClassifier` para mejorar el *fitting* del estimador.

In [87]:
estimator = GradientBoostingClassifier(loss = 'exponential', criterion = 'friedman_mse', n_estimators = 50, max_depth = 5, min_samples_split = 5, random_state = 3)
base_classifier = AdaBoostClassifier(estimator = estimator, n_estimators = 100, random_state = 3)
multi_target_classifier = MultiOutputClassifier(base_classifier, n_jobs = 4)
multi_target_classifier.fit(x_train, y_train_train)
y_pred = multi_target_classifier.predict(x_test)
report = classification_report(y_train_test, y_pred, zero_division = 0)
print(report)

              precision    recall  f1-score   support

           0       0.75      0.46      0.57        26
           1       1.00      0.83      0.91        35
           2       0.56      0.69      0.62        13
           3       0.86      0.66      0.75        29
           4       1.00      0.87      0.93        23
           5       1.00      0.94      0.97        36
           6       0.88      0.45      0.60        31
           7       1.00      0.92      0.96        25
           8       0.96      0.92      0.94        25
           9       1.00      1.00      1.00        35

   micro avg       0.93      0.78      0.85       278
   macro avg       0.90      0.77      0.82       278
weighted avg       0.92      0.78      0.84       278
 samples avg       0.84      0.78      0.79       278



##### Light Gradient-Boosting

In [18]:
base_classifier = lgb.LGBMClassifier(boosting_type = 'dart', n_estimators = 50, objective = 'binary', random_state = 3)
multi_target_classifier = MultiOutputClassifier(base_classifier, n_jobs = 4)
multi_target_classifier.fit(x_train.astype(np.float32), y_train_train)
y_pred = multi_target_classifier.predict(x_test.astype(np.float32))
report = classification_report(y_train_test, y_pred, zero_division = 0)
print(report)

              precision    recall  f1-score   support

           0       0.67      0.54      0.60        26
           1       1.00      0.74      0.85        35
           2       0.77      0.77      0.77        13
           3       0.81      0.76      0.79        29
           4       1.00      0.87      0.93        23
           5       1.00      0.89      0.94        36
           6       0.56      0.48      0.52        31
           7       1.00      0.84      0.91        25
           8       0.96      0.92      0.94        25
           9       1.00      1.00      1.00        35

   micro avg       0.89      0.78      0.83       278
   macro avg       0.88      0.78      0.82       278
weighted avg       0.89      0.78      0.83       278
 samples avg       0.82      0.77      0.77       278



## Guardar predicciones de Test

In [17]:
y_test_pred = model.predict(x_testArray)

### Etiquetado de textos sin etiqueta

#### Asignación de la etiqueta más probable

In [18]:
zeroIdx = np.where(np.sum(y_test_pred, axis = 1) == 0)[0]
if zeroIdx.size > 0:
    print('Non-labeled texts:', zeroIdx)
    probs = model.predict_proba(x_testArray)
    for idx in zeroIdx:
        oneProb = []
        for probArray, classes in zip(probs, model.classes_):
            p = probArray[idx][classes == 1][0]
            oneProb.append(p)
        y_test_pred[idx, np.argmax(oneProb)] = 1

Non-labeled texts: [ 84  85  91  92 108 138 139 140 155 168 169 176 189]


#### Active-Learning

In [ ]:
x_trainStack = x_train
y_trainStack = y_train_train
x_testStack = x_testArray
y_predStack = y_test_pred

zeroIdx = np.where(np.sum(y_predStack, axis = 1) == 0)[0]
nZeros = []
while zeroIdx.size > 0:

    model.fit(x_trainStack, y_trainStack)
    y_predStack = model.predict(x_testStack)

    zeroIdx = np.where(np.sum(y_predStack, axis = 1) == 0)[0]
    nZeros.append(zeroIdx.size)

    print(x_trainStack.shape, x_testStack.shape, nZeros)

    if len(nZeros) >= 3 and (nZeros[-1] == nZeros[-2] == nZeros[-3]):
        raise Exception('Active Learning could not label any sample in the last 2 iterations. Quitting...')

    x_lab = x_testStack[[i for i in range(x_testStack.shape[0]) if i not in zeroIdx]]
    x_testStack = x_testStack[zeroIdx]

    y_lab = y_predStack[[i for i in range(y_predStack.shape[0]) if i not in zeroIdx]]

    x_trainStack = vstack([x_trainStack, x_lab])
    y_trainStack = np.vstack([y_trainStack, y_lab])

### Escritura en fichero

In [20]:
results = pd.DataFrame(y_test_pred, index = df_test.index, columns = clases)
results.reset_index(inplace = True)

results.to_csv('results/bow-multioutput-gradientboosting.csv', index = False)
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   archivo  192 non-null    object
 1   d49      192 non-null    int32 
 2   i10      192 non-null    int32 
 3   n28      192 non-null    int32 
 4   r10      192 non-null    int32 
 5   r11      192 non-null    int32 
 6   r50      192 non-null    int32 
 7   r52      192 non-null    int32 
 8   r59      192 non-null    int32 
 9   r60      192 non-null    int32 
 10  r69      192 non-null    int32 
dtypes: int32(10), object(1)
memory usage: 9.1+ KB
